In [1]:
import polars as pl
import numpy as np
import pandas as pd
from mars.analysis.profiler import MarsDataProfiler
from mars.utils.logger import set_log_level

# 设置日志级别
set_log_level("INFO")

def get_demo_data():
    """生成一个包含各种‘脏’数据的 Demo 集，用于验证红绿灯报警逻辑"""
    np.random.seed(1234)
    rows = 10000
    
    # 1. 生成基础数据
    data = {
        "month": np.random.choice(["202401", "202402", "202403", "202404"], size=rows),
        "good_feature": np.random.normal(100, 10, size=rows),  # 完美特征
    }
    
    # 2. 注入脏数据 (测试 DQ 报警)
    
    # [Case A] 高缺失 (Missing Rate > 50%) -> show_missing 应全红
    bad_missing = np.random.normal(0, 1, size=rows)
    bad_missing[np.random.rand(rows) < 0.6] = -999 
    data["feat_high_missing"] = bad_missing
    
    # [Case B] 准常量 (Top1 Ratio > 99%) -> show_top1 应全红 [NEW]
    # 模拟：99.9% 的人都是 0，只有 0.1% 是 1
    quasi = np.zeros(rows)
    quasi[np.random.rand(rows) < 0.001] = 1
    data["feat_quasi_constant"] = quasi

    # [Case C] 高基数 (Unique Rate = 100%) -> show_unique 应全蓝 [NEW]
    # 模拟：用户 ID
    data["feat_user_id"] = [f"user_{i}" for i in range(rows)]

    # 3. 注入不稳定特征 (测试 Stats 报警)

    # [Case D] 剧烈抖动 (Stability/CV 爆炸) -> show_stats 应有红色长条
    # 模拟：前三个月均值100，第四个月均值突变成 1000
    unstable = np.random.normal(100, 5, size=rows)
    mask_apr = data["month"] == "202404"
    unstable[mask_apr] = unstable[mask_apr] * 10 
    data["feat_unstable"] = unstable

    return pl.DataFrame(data)

# 生成数据
df = get_demo_data()
print(f"✅ Demo Data Ready: {df.shape}")
print("包含测试场景: 正常, 高缺失, 准常量(Top1), 高基数(ID), 剧烈抖动(Unstable)")
df.head()

✅ Demo Data Ready: (10000, 6)
包含测试场景: 正常, 高缺失, 准常量(Top1), 高基数(ID), 剧烈抖动(Unstable)


month,good_feature,feat_high_missing,feat_quasi_constant,feat_user_id,feat_unstable
str,f64,f64,f64,str,f64
"""202404""",95.73776,-0.051205,0.0,"""user_0""",950.957766
"""202404""",109.295043,-999.0,0.0,"""user_1""",983.240373
"""202403""",104.267144,-0.809159,0.0,"""user_2""",101.073494
"""202402""",91.648176,0.277759,0.0,"""user_3""",103.256861
"""202401""",110.037528,1.419918,0.0,"""user_4""",98.321816


In [2]:
# 初始化分析器 (定义 -999 为缺失值)
profiler = MarsDataProfiler(df, custom_missing_values=[-999])

# 执行分组透视
# 我们重点关注 'mean' (均值漂移) 和 'max' (极值跳变)
report = profiler.generate_profile(profile_by="month") # 

[MARS] 2026-01-10 00:50:18 - INFO - Starting profiling (Group: month, Sparkline: True)...
[MARS] 2026-01-10 00:50:18 - INFO - Profile generated successfully.
[MARS] 2026-01-10 00:50:18 - INFO - ⏱️ [MarsDataProfiler.generate_profile] finished in 0.0208s


In [3]:
report

In [4]:
report.show_overview()

feature,dtype,distribution,missing_rate,zeros_rate,unique_rate,top1_ratio,mean,std,min,max,median,p25,p75
feat_high_missing,Float64,__▃▆█▅▂_,59.77%,0.00%,40.24%,59.77%,-597.09,489.91,-999.00,3.28,-999.00,-999.00,-0.28
feat_quasi_constant,Float64,█_______,0.00%,99.88%,0.02%,99.88%,0.00,0.03,0.00,1.00,0.00,0.00,0.00
feat_unstable,Float64,█_____▂_,0.00%,0.00%,100.00%,0.01%,327.97,392.52,81.62,1166.73,102.13,97.88,887.05
good_feature,Float64,__▃▇█▄__,0.00%,0.00%,100.00%,0.01%,100.08,10.01,61.41,136.67,100.12,93.34,106.85
feat_user_id,String,None,0.00%,0.00%,100.00%,0.01%,nan,nan,nan,nan,nan,nan,nan
month,String,None,0.00%,0.00%,0.04%,25.31%,nan,nan,nan,nan,nan,nan,nan


In [5]:
report.show_dq("missing")

feature,dtype,202401,202402,202403,202404,total
feat_high_missing,Float64,60.67%,58.39%,60.43%,59.62%,59.77%
feat_quasi_constant,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feat_unstable,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
good_feature,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feat_user_id,String,0.00%,0.00%,0.00%,0.00%,0.00%


In [6]:
report.show_dq("zeros")

feature,dtype,202401,202402,202403,202404,total
feat_high_missing,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feat_quasi_constant,Float64,99.96%,99.92%,99.76%,99.88%,99.88%
feat_unstable,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
good_feature,Float64,0.00%,0.00%,0.00%,0.00%,0.00%
feat_user_id,String,0.00%,0.00%,0.00%,0.00%,0.00%


In [7]:
report.show_dq("unique")

feature,dtype,202401,202402,202403,202404,total
feat_high_missing,Float64,39.37%,41.65%,39.61%,40.42%,40.24%
feat_quasi_constant,Float64,0.08%,0.08%,0.08%,0.08%,0.02%
feat_unstable,Float64,100.00%,100.00%,100.00%,100.00%,100.00%
good_feature,Float64,100.00%,100.00%,100.00%,100.00%,100.00%
feat_user_id,String,100.00%,100.00%,100.00%,100.00%,100.00%


In [8]:
report.show_dq("top1")

feature,dtype,202401,202402,202403,202404,total
feat_high_missing,Float64,60.67%,58.39%,60.43%,59.62%,59.77%
feat_quasi_constant,Float64,99.96%,99.92%,99.76%,99.88%,99.88%
feat_unstable,Float64,0.04%,0.04%,0.04%,0.04%,0.01%
good_feature,Float64,0.04%,0.04%,0.04%,0.04%,0.01%
feat_user_id,String,0.04%,0.04%,0.04%,0.04%,0.01%


In [9]:
report.show_trend("mean")

feature,dtype,202401,202402,202403,202404,total,group_var,group_cv
feat_high_missing,Float64,-606.08,-583.33,-603.73,-595.59,-597.09,105.5214,0.0172
feat_quasi_constant,Float64,0.00,0.00,0.00,0.00,0.00,0.0000,0.7247
feat_unstable,Float64,100.01,99.98,99.97,1000.76,327.97,202850.0319,1.3850
good_feature,Float64,100.05,100.26,99.71,100.27,100.08,0.0691,0.0026
feat_user_id,String,nan,nan,nan,nan,nan,0.0000,0.0000


In [10]:
report.show_trend("max")

feature,dtype,202401,202402,202403,202404,total,group_var,group_cv
feat_high_missing,Float64,2.90,3.28,2.97,2.77,3.28,0.0479,0.0735
feat_quasi_constant,Float64,1.00,1.00,1.00,1.00,1.00,0.0000,0.0000
feat_unstable,Float64,117.74,118.94,116.04,1166.73,1166.73,275184.9900,1.3810
good_feature,Float64,132.88,136.67,130.01,129.28,136.67,11.2464,0.0254
feat_user_id,String,nan,nan,nan,nan,nan,0.0000,0.0000


In [11]:
report.show_trend("min")

feature,dtype,202401,202402,202403,202404,total,group_var,group_cv
feat_high_missing,Float64,-999.00,-999.00,-999.00,-999.00,-999.00,0.0000,0.0000
feat_quasi_constant,Float64,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000
feat_unstable,Float64,81.62,83.30,82.32,802.13,81.62,129499.1790,1.3717
good_feature,Float64,64.01,61.41,65.65,61.82,61.41,3.9172,0.0313
feat_user_id,String,nan,nan,nan,nan,nan,0.0000,0.0000


In [12]:
report.write_excel("demo_report.xlsx")

[MARS] 2026-01-10 00:50:18 - INFO - 📊 Exporting to demo_report.xlsx...
[MARS] 2026-01-10 00:50:18 - INFO - ✅ Done.


In [13]:
report.get_profile_data()

(shape: (6, 14)
 ┌────────────┬─────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
 │ feature    ┆ dtype   ┆ distribut ┆ missing_r ┆ … ┆ max       ┆ median    ┆ p25       ┆ p75       │
 │ ---        ┆ ---     ┆ ion       ┆ ate       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
 │ str        ┆ str     ┆ ---       ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64       │
 │            ┆         ┆ str       ┆ f64       ┆   ┆           ┆           ┆           ┆           │
 ╞════════════╪═════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
 │ feat_high_ ┆ Float64 ┆ __▃▆█▅▂_  ┆ 0.5977    ┆ … ┆ 3.28316   ┆ -999.0    ┆ -999.0    ┆ -0.275044 │
 │ missing    ┆         ┆           ┆           ┆   ┆           ┆           ┆           ┆           │
 │ feat_quasi ┆ Float64 ┆ █_______  ┆ 0.0       ┆ … ┆ 1.0       ┆ 0.0       ┆ 0.0       ┆ 0.0       │
 │ _constant  ┆         ┆           ┆           ┆   ┆           ┆ 

In [14]:
# 伪代码逻辑
def filter_bad_features(df):
    
    # 1. 删掉“太无聊”的列 (准常量)
    # 只要 Top1 占比太高，不管它是啥类型，直接删
    top1_ratio = 0.96
    if top1_ratio > 0.95: 
        return "DROP (Quasi-Constant)"

    # 2. 删掉“太杂乱”的 ID 列 (高基数)
    # 只有当它是字符串类型，且几乎都不重复时，才删
    # (数值型的高Unique是好事，不能删！)
    unique_rate = 1.0
    is_string_type = True
    if unique_rate > 0.95 and is_string_type:
        return "DROP (ID Column)"
        
    return "KEEP"